In [1]:
import re, os, sys, unicodedata
import pandas as pd
import numpy as np
import py_stringmatching as sm
import codecs

In [2]:
def _normalize(data):
    """ Normalize unicode data in Python to remove umlauts, accents, spaces, etc."""
    normal = unicodedata.normalize('NFKD', data).encode('ASCII', 'ignore')
    val = normal.decode("utf-8")
    val = val.lower()
    # remove special characters
    val = re.sub('[^A-Za-z]+', '', val)
    return val

In [3]:
path = '/Users/galvanize/infosys/cip/data/nc_data'
filename0, filename1 = 'nc0.csv', 'nc1.csv'
filepath0, filepath1 = os.path.join(path, filename0), os.path.join(path, filename1)

In [33]:
df0 = pd.read_csv(filepath0, dtype={'recid':np.int64, 'givenname':str, 'surname':str, 'suburb':str, 'postcode':str})
df1 = pd.read_csv(filepath1, dtype={'recid':np.int64, 'givenname':str, 'surname':str, 'suburb':str, 'postcode':str})


In [9]:
df0.info(), df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 5 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   recid      1000000 non-null  int64 
 1   givenname  999928 non-null   object
 2   surname    999980 non-null   object
 3   suburb     999953 non-null   object
 4   postcode   999942 non-null   object
dtypes: int64(1), object(4)
memory usage: 38.1+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 5 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   recid      1000000 non-null  int64 
 1   givenname  999942 non-null   object
 2   surname    999983 non-null   object
 3   suburb     999951 non-null   object
 4   postcode   999943 non-null   object
dtypes: int64(1), object(4)
memory usage: 38.1+ MB


(None, None)

In [34]:
df0.columns= [f'{col}0' for col in df0.columns]
df1.columns = [f'{col}1' for col in df1.columns]

df = pd.concat([df0,df1], axis=1)
df = df[['recid0', 'recid1','suburb0','suburb1','givenname0','givenname1','surname0','surname1','postcode0','postcode1' ]]

In [35]:
answers = df[['recid0', 'recid1']]
df = df.drop(['recid0', 'recid1'], axis=1)

In [36]:
# str() conversion seems superfluous here (is also slow!) but there is some weird data that throws errors in the dataset w/o it 
for col in df.columns:
    df.loc[:,col] = df[col].apply(lambda x: str(x)).str.replace('[^A-Za-z0-9]+', '', regex=True)
df.head()

,suburb0,suburb1,givenname0,givenname1,surname0,surname1,postcode0,postcode1
0,waxhaw,raleigh,kadelyn,ashlev,gragnani,batts,2873,276o6
1,washington,vass,ronel,tiffamy,carter,perez,2788g,28354
2,millersceeek,charlogte,jason,david,roehrig,murry,286s1,28268
3,greensboro,rockymount,antrea,shamkea,mure,hutchinson,27410,278o4
4,jacksonville,louisburg,caedar,john,oliver,snyddr,28542,27s49


df['suburb0'].apply(lambda x: set(tok.tokenize(x)))

In [37]:
# this cell is crazy slow
tok = sm.QgramTokenizer(qval=3, return_set=True, padding=False)
for col in df.columns:
    df.loc[:,f'{col}_qgrams'] = df[col].apply(lambda x: tok.tokenize(x))
df.head()

,suburb0,suburb1,givenname0,givenname1,surname0,surname1,postcode0,postcode1,suburb0_qgrams,suburb1_qgrams,givenname0_qgrams,givenname1_qgrams,surname0_qgrams,surname1_qgrams,postcode0_qgrams,postcode1_qgrams
0,waxhaw,raleigh,kadelyn,ashlev,gragnani,batts,2873,276o6,"[wax, axh, xha, haw]","[ral, ale, lei, eig, igh]","[kad, ade, del, ely, lyn]","[ash, shl, hle, lev]","[gra, rag, agn, gna, nan, ani]","[bat, att, tts]","[287, 873]","[276, 76o, 6o6]"
1,washington,vass,ronel,tiffamy,carter,perez,2788g,28354,"[was, ash, shi, hin, ing, ngt, gto, ton]","[vas, ass]","[ron, one, nel]","[tif, iff, ffa, fam, amy]","[car, art, rte, ter]","[per, ere, rez]","[278, 788, 88g]","[283, 835, 354]"
2,millersceeek,charlogte,jason,david,roehrig,murry,286s1,28268,"[mil, ill, lle, ler, ers, rsc, sce, cee, eee, ...","[cha, har, arl, rlo, log, ogt, gte]","[jas, aso, son]","[dav, avi, vid]","[roe, oeh, ehr, hri, rig]","[mur, urr, rry]","[286, 86s, 6s1]","[282, 826, 268]"
3,greensboro,rockymount,antrea,shamkea,mure,hutchinson,27410,278o4,"[gre, ree, een, ens, nsb, sbo, bor, oro]","[roc, ock, cky, kym, ymo, mou, oun, unt]","[ant, ntr, tre, rea]","[sha, ham, amk, mke, kea]","[mur, ure]","[hut, utc, tch, chi, hin, ins, nso, son]","[274, 741, 410]","[278, 78o, 8o4]"
4,jacksonville,louisburg,caedar,john,oliver,snyddr,28542,27s49,"[jac, ack, cks, kso, son, onv, nvi, vil, ill, ...","[lou, oui, uis, isb, sbu, bur, urg]","[cae, aed, eda, dar]","[joh, ohn]","[oli, liv, ive, ver]","[sny, nyd, ydd, ddr]","[285, 854, 542]","[27s, 7s4, s49]"


In [38]:
from collections import defaultdict, namedtuple

total = set()
f = [{4,5}, {5,6}, {6,8}, {9,4}]
for item in f:
    total.update(item)

total

task: 
    find all the existing trigrams for each field


In [42]:
all_suburb_qgrams = set().union(*df['suburb0_qgrams'].values).union(*df['suburb1_qgrams'])
all_givenname_qgrams = set().union(*df['givenname0_qgrams'].values).union(*df['givenname1_qgrams'])
all_surname_qgrams = set().union(*df['surname0_qgrams'].values).union(*df['surname1_qgrams'])
all_postcode_qgrams = set().union(*df['postcode0_qgrams'].values).union(*df['postcode1_qgrams'])
len(all_suburb_qgrams), len(all_givenname_qgrams), len(all_surname_qgrams), len(all_postcode_qgrams)

(12536, 13470, 16608, 2199)

In [ ]:
task: 
    for each column 
    assign the column index to trigrams it contains 

In [45]:
## super slow!!!!
# suburbs key = trigram, value = index

suburb_dd = defaultdict(list)
for value, keys in zip(df.index, df['suburb0_qgrams']):
    for key in keys:
        suburb_dd[key].append(['0',value])

for value, keys in zip(df.index, df['suburb1_qgrams']):
    for key in keys:
        suburb_dd[key].append(['1',value])

In [47]:
len(suburb_dd['wax'])

9229

In [48]:
givenname_dd =  defaultdict(list)
for value, keys in zip(df.index, df['givenname0_qgrams']):
    for key in keys:
        givenname_dd[key].append(['0',value])

for value, keys in zip(df.index, df['givenname1_qgrams']):
    for key in keys:
        givenname_dd[key].append(['1',value])

In [49]:
surname_dd = defaultdict(list)
for value, keys in zip(df.index, df['surname0_qgrams']):
    for key in keys:
        surname_dd[key].append(['0',value])

for value, keys in zip(df.index, df['surname1_qgrams']):
    for key in keys:
        surname_dd[key].append(['1',value])

In [50]:
postcode_dd = defaultdict(list)
for value, keys in zip(df.index, df['postcode0_qgrams']):
    for key in keys:
        postcode_dd[key].append(['0',value])

for value, keys in zip(df.index, df['postcode1_qgrams']):
    for key in keys:
        postcode_dd[key].append(['1',value])

In [51]:
!ls

data                 readme.md
nc_test.ipynb        record_linkage.ipynb


In [52]:
import pickle

In [53]:
saved_data = {}
saved_data['suburb_dd']= suburb_dd
saved_data['givenname_dd']= givenname_dd
saved_data['surname_dd']= surname_dd
saved_data['postcode_dd']= postcode_dd

saved_data['all_suburb_qgrams']= all_suburb_qgrams
saved_data['all_givenname_qgrams']= all_givenname_qgrams
saved_data['all_surname_qgrams']= all_surname_qgrams
saved_data['all_postcode_qgrams']= all_postcode_qgrams

saved_data['df']= df
saved_data['df0']= df0
saved_data['df1']= df1

with open('nc_test.pickle', 'wb') as f:
    pickle.dump(saved_data, f)

In [54]:
computed_items = {}
computed_items['suburb_dd']= suburb_dd
computed_items['givenname_dd']= givenname_dd
computed_items['surname_dd']= surname_dd
computed_items['postcode_dd']= postcode_dd

computed_items['all_suburb_qgrams']= all_suburb_qgrams
computed_items['all_givenname_qgrams']= all_givenname_qgrams
computed_items['all_surname_qgrams']= all_surname_qgrams
computed_items['all_postcode_qgrams']= all_postcode_qgrams

with open('nc_test_computed.pickle', 'wb') as f:
    pickle.dump(computed_items, f)
    
data_frame = {}
data_frame['df']= df
with open('nc_test_df.pickle', 'wb') as f:
    pickle.dump(data_frame, f)

In [ ]:
# assign trigrams for each r
dd = defaultdict(list)

for value, keys in zip(df.index, df.set_of_trigrams):
    for key in keys:
        dd[key].append(value)

In [119]:
QGRAMS = namedtuple('QGRAMS','suburb, givenname, surname, postcode') 

In [123]:
data0 = QGRAMS(df['suburb0_qgrams'],df['givenname0_qgrams'],df['surname0_qgrams'],df['postcode0_qgrams'])

In [124]:
data1 = QGRAMS(df['suburb1_qgrams'],df['givenname1_qgrams'],df['surname1_qgrams'],df['postcode1_qgrams'])

In [126]:
data0.suburb

0                                      {xha, haw, wax, axh}
1                  {shi, gto, ton, ngt, ing, ash, was, hin}
2         {eee, ill, mil, ler, ers, sce, cee, lle, eek, ...
3                  {ens, gre, ree, sbo, nsb, oro, bor, een}
4         {cks, ill, nvi, vil, onv, son, kso, ack, lle, ...
                                ...                        
999995                  {har, ott, rlo, arl, lot, cha, tte}
999996                                           {den, ede}
999997    {pri, ows, ill, wsp, wil, ngs, ing, spr, rin, ...
999998                            {rem, mon, ont, fre, emo}
999999                       {erd, dee, rde, abe, ber, een}
Name: suburb0_qgrams, Length: 1000000, dtype: object

In [127]:
data1.suburb

0                        {lei, ral, ale, eig, igh}
1                                       {vas, ass}
2              {har, rlo, arl, gte, ogt, log, cha}
3         {ock, roc, kym, mou, unt, ymo, oun, cky}
4              {uis, oui, isb, lou, bur, sbu, urg}
                            ...                   
999995                   {lei, ral, ale, eig, igh}
999996    {min, wil, gto, ton, ngt, ing, lmi, ilm}
999997                        {ngi, gie, ier, ang}
999998                   {mci, lmc, cit, ity, elm}
999999         {ann, ner, rel, ban, nne, ere, elk}
Name: suburb1_qgrams, Length: 1000000, dtype: object